In [ ]:
import pandas as pd
import numpy as np
import sys,os
import matplotlib.pyplot as plt
import concurrent.futures
import importlib
import seaborn as sns

import plot_loaders
import plot_utils

In [ ]:
ORDERINGS = {
    "noise_level" : [x[0] for x in sorted(plot_loaders.NOISES_MAPPING.items(), key= lambda x:x[1]) ],
    "variant":["nonoise","zerosum","muffliato"],
    "avgsteps": ["10avgsteps", "1avgsteps"],
    "topology_type": ["static","dynamic"],
} 

#CIFAR Additional runs with seeds:
# target_dir = "my_results/icml_experiments/additional_cifar10_v2/"
# ORDERINGS = {
#     "noise_level" : ["nonoise","4th","64th"],
#     "variant":["nonoise","zerosum","muffliato"],
#     "avgsteps": ["10avgsteps", "1avgsteps"],
#     "topology_type": ["static","dynamic"],
# } 


# Muffliato experiments on CIFAR with multiple avg steps
# target_dir = "my_results/icml_experiments/muffliato_avgsteps"
# ORDERINGS = {
#     "noise_level" : ["4th","16th","64th"],
#     "variant":["muffliato","zerosum"],
#     "avgsteps": ["20avgsteps", "15avgsteps", "10avgsteps","5avgsteps", "1avgsteps"].reverse(),
#     "topology_type": ["static","dynamic"],
# }


TOTAL_PROCESSES = 128
MAX_MACHINES =  4

# S&P Femnist experiments
# target_dir = "my_results/femnist_10shards_correctedrandomness/"


# TOTAL_PROCESSES = 64
# MAX_MACHINES =  4
# MAX_ITERATIONS=5000
# STARTING_ITERATION = 0

# MovieLens Experiments
# target_dir = "my_results/movielens_full/"

#Renewed CIFAR
target_dir= "my_results/cifar_longer/"

linestyles = {
    "ZeroSum" : "--"
}

fontsize=20

alpha = 0.1

save_directory = "assets/"

assert TOTAL_PROCESSES%MAX_MACHINES == 0
MAX_PROCESSES = TOTAL_PROCESSES//MAX_MACHINES

In [ ]:
# Load the file list
experiments_dict = plot_loaders.get_full_path_dict(target_dir)
experiments_attributes = plot_loaders.get_experiments_dict(experiments_dict)

experiments_attributes_df = pd.DataFrame({})
for experiment_name,experiment_attribute in sorted(experiments_attributes.items()):
    print(experiment_name)
    experiments_attributes_df = pd.concat([experiments_attributes_df,pd.DataFrame(experiment_attribute,index=[experiment_name])])
# print(experiments_attributes_df.columns)

# print(experiments_attributes["4447548_muffliato_128nodes_10avgsteps_64th_dynamic_seed105"])
print(experiments_dict)
# for seed in range(91,105): 
#     print(f"Seed: {seed:3.0f}:", end= "")
#     current_seed = experiments_attributes_df[experiments_attributes_df["seed"] == f"seed{seed}"]
#     current_variants = list(current_seed["variant"])
#     current_topologies = list(current_seed["topology_type"])
#     print(sorted(list(zip(current_variants,current_topologies))))


In [ ]:
importlib.reload(plot_loaders)
loaded_data = {}


with concurrent.futures.ProcessPoolExecutor(max_workers=20) as executor:
    futures = {}
    for name,filepath in sorted(experiments_dict.items()):
        # plot_loaders.load_data_element(filepath)
        futures[name] = executor.submit(
            plot_loaders.load_data_element, filepath
        )
    for name, result in futures.items():
        print(f"Adding data from {name}")
        res = result.result()
        if res.empty:
            print(f"Empty result for {name}")
        loaded_data[name]=res
# print(loaded_data)
print("Finished loading all data")
for column in loaded_data[name]:
    print(column)

## Agregate data

In [ ]:
new_data = {}
new_experiments_attributes = {}
for experiment_name,experiment_data in loaded_data.items():
    if experiment_data.empty:
        print(f"Deleting {experiment_name} as it did not load")
        continue
    new_data[experiment_name] = experiment_data
    new_experiments_attributes[experiment_name] = experiments_attributes[experiment_name]
loaded_data = new_data
experiments_attributes=new_experiments_attributes
del new_data
del new_experiments_attributes


importlib.reload(plot_loaders)
# For most of the attacks
general_agg_methods = ["mean", "std", "sum","min","max",plot_loaders.percentile(0.5),plot_loaders.percentile(0.95)]
# columns_to_agg = ["train_loss","test_loss", "test_acc", "total_bytes", "total_meta", "total_data_per_n", "roc_auc","roc_auc_balanced","50auc-distance","50auc-distance_balanced"]
columns_to_agg = ["train_loss","test_loss", "test_acc", "total_bytes", "total_meta", "total_data_per_n"]

# columns_to_agg += ["classifier_roc_auc", ]
# columns_to_agg += [f"classifier_tpr_at_fpr{fpr}" for fpr in [1e-5,1e-4,1e-3,1e-2,1e-1]]

linkability_aggregators = {
    "linkability_top1": [plot_loaders.count_percentage_success],
    "linkability_top5": [plot_loaders.count_percentage_success],
    "linkability_real_rank": ["median","min","max"],
}
linkability_aggregators = {} # Remember to comment this line if you want linkability results.

to_start_avg = ["roc_auc mean","linkability_top1 count_percentage_success","roc_auc max"]
to_start_avg = []
general_aggregator = {column:general_agg_methods for column in columns_to_agg}

general_aggregator.update(linkability_aggregators) 

formatted_data = {}
for name,data in sorted(loaded_data.items()):
    print(f"Formatting {name}")
    formatted_data[name] = plot_loaders.format_data(data,name,columns_to_agg,linkability_aggregators,general_aggregator,TOTAL_PROCESSES,to_start_avg)
formatted_data[list(formatted_data)[0]]

In [ ]:
formatted_data[list(formatted_data)[0]]["classifier_attacker_fraction"]

In [ ]:
importlib.reload(plot_loaders)
importlib.reload(plot_utils)
static_filter = {"topology_type" : ["static"] }
dynamic_filter = {"topology_type" : ["dynamic"]}

static_experiments =  plot_loaders.filter_attribute(experiments_attributes,static_filter)
dynamic_experiments = plot_loaders.filter_attribute(experiments_attributes,dynamic_filter)

#To display full experiments
display_attributes = {
    "hue": "noise_level",
    "style":["variant","avgsteps","additional_attribute"],
    "col" : "topology_type",
}

# To display experiment parameters tuning.
# display_attributes = {
#     "hue": "lr",
#     "style":"local_rounds",
#     "col" : "model",
# }

# To display muffliato results
# display_attributes = {
#     "hue":  "avgsteps",
#     "style":"variant",
#     "col" : "noise_level",
# }

for experiment in experiments_attributes:
    print(experiment)

plot_utils.plot_all_experiments(
    formatted_data,
    experiments_attributes,
    display_attributes,
    "All experiments accuracies evolution",
    "test_acc mean",
    save_directory=save_directory,
    orderings = ORDERINGS,
)


## Noise cancellation vs no noise cancellation

In [ ]:

importlib.reload(plot_utils)
zerosum_attributes = {"variant":["nonoise","zerosum"]}
# zerosum_attributes = {"variant":["nonoise","zerosum"],"topology_type":["static"]}
# zerosum_attributes = {"variant":["nonoise","zerosum"],"topology_type":["dynamic"]}

display_attributes = {
    "hue":"noise_level",
    "style": "additional_attribute",
    "size": "variant",
    "col":"topology_type"
}

zerosum_experiments = plot_loaders.filter_attribute(experiments_attributes,zerosum_attributes)
for experiment in sorted(zerosum_experiments):
    print(experiment)

plot_utils.plot_all_experiments(
    formatted_data,
    zerosum_experiments,
    display_attributes,
    "Test accuracy evolution of ZeroSum with and without self noise",
    "test_acc mean",
    save_directory=save_directory,
    orderings=ORDERINGS,
)



# Attack results
## In the static case:

In [ ]:
importlib.reload(plot_utils)
zerosum_attributes = {"variant":["nonoise","zerosum"]}
# zerosum_attributes = {"variant":["nonoise","zerosum"],"topology_type":["static"]}
# zerosum_attributes = {"variant":["nonoise","zerosum"],"topology_type":["dynamic"]}

display_attributes = {
    "hue":"noise_level",
    "style": "additional_attribute",
    "size": "variant",
    "col": "topology_type",
}

zerosum_experiments = plot_loaders.filter_attribute(experiments_attributes,zerosum_attributes)
for experiment in sorted(zerosum_experiments):
    print(experiment)

plot_utils.plot_all_experiments(
    formatted_data,
    zerosum_experiments,
    display_attributes,
    "Attack AUC of Zerosum with and without self noise",
    "roc_auc mean",
    save_directory=save_directory,
    orderings=ORDERINGS,
)

# Results vs Muffliato

In [ ]:
importlib.reload(plot_utils)

zerosum_attributes = {
    "variant":["zerosum","nonoise"],
    # "topology_type":["static"],
    # "topology_type":["dynamic"],
    "avgsteps": ["1avgsteps"],
    # "avgsteps": ["1avgsteps","10avgsteps"],
    "additional_attribute": ["nonoise","selfnoise"]}
muffliato_attributes = {
    "variant":["muffliato"],
    # "topology_type":["static"],
    # "topology_type":["dynamic"],
    "avgsteps":["10avgsteps"],
}

current_attributes= [zerosum_attributes,muffliato_attributes]

# display_attributes = {
#     "hue":"noise_level",
#     "style":"variant",
#     # "size":"variant", # For some reason having both style and size breaks in this case.
#     "col":"topology_type",
# }

display_attributes = {
    "hue":["noise_level", "variant"],
    "style":["noise_level", "variant"],
    # "size":"variant", # For some reason having both style and size breaks in this case.
    "col":"topology_type",
}

zerosum_experiments = plot_loaders.filter_attribute_list(experiments_attributes,current_attributes)
for experiment in sorted(zerosum_experiments):
    print(experiment)


plot_utils.plot_all_experiments(
    formatted_data,
    zerosum_experiments,
    display_attributes,
    "Accuracy of Zerosum and Muffliato",
    "test_acc mean",
    save_directory=save_directory,
    orderings=ORDERINGS,
)

plt.figure()
plot_utils.plot_all_experiments(
    formatted_data,
    zerosum_experiments,
    display_attributes,
    "Attack AUC of Zerosum and Muffliato",
    "roc_auc mean",
    save_directory=save_directory,
    orderings=ORDERINGS,
)

### Smaller plots to have an easier time to read the information

In [ ]:
importlib.reload(plot_utils)
zerosum_attributes = {
    "variant":["nonoise","zerosum"],
    # "topology_type":["dynamic"],
    "avgsteps": ["1avgsteps"], 
    "additional_attribute": ["nonoise","selfnoise"],
    "noise_level":["nonoise","1th","2th","4th"]
}

muffliato_attributes = {
    "variant":["muffliato"],
    # "topology_type":["dynamic"],
    "avgsteps": ["10avgsteps"], 
    "noise_level":["1th","2th","4th"]
}
current_attributes= [zerosum_attributes,muffliato_attributes]


# display_attributes = {
#     "hue":"noise_level",
#     "style":"variant",
#     # "size":"variant", # For some reason having both style and size breaks in this case.
#     "col":"topology_type",
# }

display_attributes = {
    "hue":["noise_level", "variant"],
    "style":["noise_level", "variant"],
    # "size":"variant", # For some reason having both style and size breaks in this case.
    "col":"topology_type",
}

zerosum_experiments = plot_loaders.filter_attribute_list(experiments_attributes,current_attributes)
for experiment in sorted(zerosum_experiments):
    print(experiment)



plot_utils.plot_all_experiments(
    formatted_data,
    zerosum_experiments,
    display_attributes,
    "High noise:Accuracy of Zerosum and Muffliato with and without self noise",
    "test_acc mean",
    save_directory=save_directory,
    orderings=ORDERINGS,
)

plt.figure()

plot_utils.plot_all_experiments(
    formatted_data,
    zerosum_experiments,
    display_attributes,
    "High noise:Attack AUC of Zerosum and Muffliato with and without self noise",
    "roc_auc max",
    save_directory=save_directory,
    orderings=ORDERINGS,
)
plt.ylim(top=1)

# Plot the middle noises
importlib.reload(plot_utils)
zerosum_attributes = {
    "variant":["nonoise","zerosum"],
    # "topology_type":["dynamic"],
    "avgsteps": ["1avgsteps"], 
    "additional_attribute": ["nonoise","selfnoise"],
    "noise_level":["nonoise","8th","16th"]
}

muffliato_attributes = {
    "variant":["muffliato"],
    # "topology_type":["dynamic"],
    "avgsteps": ["10avgsteps"], 
    "noise_level":["8th","16th"]
}
current_attributes= [zerosum_attributes,muffliato_attributes]
zerosum_experiments = plot_loaders.filter_attribute_list(experiments_attributes,current_attributes)

plt.figure()
plot_utils.plot_all_experiments(
    formatted_data,
    zerosum_experiments,
    display_attributes,
    "Middle noise:Accuracy of Zerosum and Muffliato with and without self noise",
    "test_acc mean",
    save_directory=save_directory,
    orderings=ORDERINGS,
)

plt.figure()
plot_utils.plot_all_experiments(
    formatted_data,
    zerosum_experiments,
    display_attributes,
    "Middle noise:Attack AUC of Zerosum and Muffliato with and without self noise",
    "roc_auc max",
    save_directory=save_directory,
    orderings=ORDERINGS,
)
plt.ylim(top=1)

# Plot the low noises
importlib.reload(plot_utils)
zerosum_attributes = {
    "variant":["nonoise","zerosum"],
    # "topology_type":["dynamic"],
    "avgsteps": ["1avgsteps"], 
    "additional_attribute": ["nonoise","selfnoise"],
    "noise_level":["nonoise","32th","64th","128th"]
}

muffliato_attributes = {
    "variant":["muffliato"],
    # "topology_type":["dynamic"],
    "avgsteps": ["10avgsteps"], 
    "noise_level":["32th","64th","128th"]
}
current_attributes= [zerosum_attributes,muffliato_attributes]
zerosum_experiments = plot_loaders.filter_attribute_list(experiments_attributes,current_attributes)

plt.figure()
plot_utils.plot_all_experiments(
    formatted_data,
    zerosum_experiments,
    display_attributes,
    "Low noise:Accuracy of Zerosum and Muffliato with and without self noise",
    "test_acc mean",
    save_directory=save_directory,
    orderings=ORDERINGS,
)

plt.figure()
plot_utils.plot_all_experiments(
    formatted_data,
    zerosum_experiments,
    display_attributes,
    "Low noise:Attack AUC of Zerosum and Muffliato with and without self noise",
    "roc_auc max",
    save_directory=save_directory,
    orderings=ORDERINGS
)
plt.ylim(top=1)



# Trying out scatterplots

## Aggregated plots:

In [ ]:
importlib.reload(plot_utils)
x_method = "mean"
y_method = "max"
# y_method = "mean"
# y_method = "min"

zerosum_attributes = {
    "variant":["nonoise","zerosum"],
    "topology_type":["static"],
    "avgsteps": ["1avgsteps"], 
    "additional_attribute": ["nonoise","selfnoise"],
}

muffliato_attributes = {
    "variant":["muffliato"],
    "topology_type":["static"],
    "avgsteps": ["5avgsteps"],
}

name_formater = {
    "test_acc mean max":"Maximum test accuracy [%]",
    "nonoise":"test"
}

current_attributes = [zerosum_attributes,muffliato_attributes]

display_attributes = {
    "hue":"variant",
    "style":"variant",
    "size":"variant", # For some reason having both style and size breaks in this case.
    # "col":"topology_type",# Cannot have col argument for the lineplot function
}
zerosum_experiments = plot_loaders.filter_attribute_list(experiments_attributes,current_attributes)
for experiment in sorted(zerosum_experiments):
    print(experiment)


print([x for x in formatted_data[experiment].columns if "test_acc" in x])
plot_utils.scatter_averaged_experiments(
    formatted_data,
    zerosum_experiments,
    display_attributes,
    "Accuracy vs AUC evolution accross entire runs-static topology",
    "test_loss mean",
    # x_axis_name="50auc-distance mean",
    x_axis_name="roc_auc mean",
    save_directory=save_directory,
    y_method=y_method,
    x_method=x_method,
    orderings=ORDERINGS,
    name_formater=name_formater,
)

# plt.figure()
# plot_utils.scatter_averaged_experiments(
#     formatted_data,
#     zerosum_experiments,
#     display_attributes,
#     "Accuracy vs Linkability attack for the entire run-static topology",
#     y_axis_name="test_loss mean",
#     x_axis_name="linkability_top1 count_percentage_success",
#     save_directory=save_directory,
#     y_method=y_method,
#     x_method=x_method,
#     orderings=ORDERINGS,
#     name_formater=name_formater,
# )

plt.figure()
plot_utils.scatter_averaged_experiments(
    formatted_data,
    zerosum_experiments,
    display_attributes,
    "Classifier attack for the entire run-static topology",
    y_axis_name="test_loss mean",
    x_axis_name="classifier_roc_auc mean",
    save_directory=save_directory,
    y_method=y_method,
    x_method=x_method,
    orderings=ORDERINGS,
    name_formater=name_formater,
)
plt.figure()
plot_utils.scatter_averaged_experiments(
    formatted_data,
    zerosum_experiments,
    display_attributes,
    "Classifier attack for the entire run-static topology",
    y_axis_name="test_loss mean",
    x_axis_name="classifier_tpr_at_fpr0.1 mean",
    save_directory=save_directory,
    y_method=y_method,
    x_method=x_method,
    orderings=ORDERINGS,
    name_formater=name_formater,
)


In [ ]:
# Look at the dynamic case
zerosum_attributes = {
    "variant":["nonoise","zerosum"],
    "topology_type":["dynamic"],
    "avgsteps": ["1avgsteps"], 
    "additional_attribute": ["nonoise","selfnoise"],
}

muffliato_attributes = {
    "variant":["muffliato"],
    "topology_type":["dynamic"],
    "avgsteps": ["10avgsteps"],
}
current_attributes = [zerosum_attributes,muffliato_attributes]
zerosum_experiments = plot_loaders.filter_attribute_list(experiments_attributes,current_attributes)

plt.figure()
plot_utils.scatter_averaged_experiments(
    formatted_data,
    zerosum_experiments,
    display_attributes,
    "Accuracy vs AUC evolution accross entire runs-dynamic topology",
    "test_acc mean",
    "roc_auc mean",
    save_directory=save_directory,
    y_method=y_method,
    x_method=x_method,
    orderings=ORDERINGS,
    name_formater=name_formater,
)

plt.figure()
plot_utils.scatter_averaged_experiments(
    formatted_data,
    zerosum_experiments,
    display_attributes,
    "Accuracy vs Linkability attack for the entire run-dynamic topology",
    y_axis_name="test_acc mean",
    x_axis_name="linkability_top1 count_percentage_success",
    save_directory=save_directory,
    y_method=y_method,
    x_method=x_method,
    orderings=ORDERINGS,
    name_formater=name_formater,
)

plt.figure()
plot_utils.scatter_averaged_experiments(
    formatted_data,
    zerosum_experiments,
    display_attributes,
    "Accuracy vs Linkability top5 attack for the entire run-dynamic topology",
    y_axis_name="test_acc mean",
    x_axis_name="linkability_top5 count_percentage_success",
    save_directory=save_directory,
    y_method=y_method,
    x_method=x_method,
    orderings=ORDERINGS,
    name_formater=name_formater,
)

In [ ]:
importlib.reload(plot_utils)
zerosum_attributes = {
    "variant":["nonoise"],
    "topology_type":["static"],
    "avgsteps": ["1avgsteps"],
    # "noise_level":[None,"2th","4th"]
}


zerosum_experiments = plot_loaders.filter_attribute(experiments_attributes,zerosum_attributes)
for experiment in sorted(zerosum_experiments):
    print(experiment)


experiment = zerosum_experiments[0]
experiment_data = formatted_data[experiment] 

sns.set_theme()
sns.relplot(data=experiment_data, x="roc_auc mean", y="test_acc mean", hue="iteration")

title = "Evolution of the unnoised test accuracy and AUC-static topology"
plt.title(title)
plt.savefig(save_directory + title.replace(" ","_") +".pdf")

## Better experiment display (selected noises levels)

In [ ]:
importlib.reload(plot_utils)
zerosum_attributes = {
    "variant":["nonoise","zerosum"],
    # "topology_type":["dynamic"],
    "avgsteps": ["1avgsteps"], 
    "additional_attribute": ["nonoise","selfnoise"],
    "noise_level":["nonoise","4th"]
}

muffliato_attributes = {
    "variant":["muffliato"],
    # "topology_type":["dynamic"],
    "avgsteps": ["10avgsteps"], 
    "noise_level":["64th"]
}
current_attributes= [zerosum_attributes,muffliato_attributes]

display_attributes = {
    "hue":["variant","noise_level","avgsteps"],
    "style":["variant","noise_level","avgsteps"],
    "size":["variant","noise_level","avgsteps"], # For some reason having both style and size breaks in this case.
    "col":"topology_type",
}

order_mapper = {"muffliato":2,"nonoise":0,"zerosum":1}
zerosum_experiments = plot_loaders.filter_attribute_list(experiments_attributes,current_attributes)
zerosum_experiments = sorted(zerosum_experiments, key = lambda x : order_mapper[x.split("_")[1]])
for experiment in zerosum_experiments:
    print(experiment)
# zerosum_experiments = [zerosum_experiments[i] for i in [0,1,4,5,2,3] ]



plot_utils.plot_all_experiments(
    formatted_data,
    zerosum_experiments,
    display_attributes,
    "Selected noise:Accuracy of Zerosum and Muffliato",
    "test_acc mean",
    save_directory=save_directory,
    orderings = ORDERINGS,
)


plt.figure()

plot_utils.plot_all_experiments(
    formatted_data,
    zerosum_experiments,
    display_attributes,
    "Selected noise:Attack AUC of Zerosum and Muffliato",
    "roc_auc mean_start_avg",
    save_directory=save_directory,
    orderings = ORDERINGS,
)

plt.figure()

plot_utils.plot_all_experiments(
    formatted_data,
    zerosum_experiments,
    display_attributes,
    "Selected noise:Linkability attack ac of Zerosum and Muffliato",
    "linkability_top1 count_percentage_success_start_avg",
    save_directory=save_directory,
    orderings = ORDERINGS,
)

plt.figure()

#Needs to only have hue since we can't have other attributes in jointplots
display_attributes = {
    "hue":["variant","noise_level","avgsteps"],
}

zerosum_attributes = {
    "variant":["nonoise","zerosum"],
    "topology_type":["static"],
    "avgsteps": ["1avgsteps"], 
    "additional_attribute": ["nonoise","selfnoise"],
    "noise_level":["nonoise","4th"]
}

muffliato_attributes = {
    "variant":["muffliato"],
    "topology_type":["static"],
    "avgsteps": ["10avgsteps"], 
    "noise_level":["64th"]
}
current_attributes= [zerosum_attributes,muffliato_attributes]

zerosum_experiments = plot_loaders.filter_attribute_list(experiments_attributes,current_attributes)
for experiment in sorted(zerosum_experiments):
    print(experiment)
print("-"*40)


plot_utils.scatter_all_experiments(
    formatted_data,
    zerosum_experiments,
    display_attributes,
    "Selected noise:Accuracy vs AUC-static",
    "roc_auc mean",
    "test_acc mean",
    save_directory=save_directory,
    orderings=ORDERINGS,
)
plt.figure()
plot_utils.scatter_all_experiments(
    formatted_data,
    zerosum_experiments,
    display_attributes,
    "Selected noise:Accuracy vs Linkability sucess rate-static",
    "linkability_top1 count_percentage_success",
    "test_acc mean",
    save_directory=save_directory,
    orderings=ORDERINGS,
)

In [ ]:
importlib.reload(plot_utils)
zerosum_attributes = {
    "variant":["nonoise","zerosum"],
    # "topology_type":["dynamic"],
    "avgsteps": ["1avgsteps"], 
    "additional_attribute": ["nonoise","selfnoise"],
    "noise_level":["nonoise","1th","2th","4th"]
}

muffliato_attributes = {
    "variant":["muffliato"],
    # "topology_type":["dynamic"],
    "avgsteps": ["10avgsteps"], 
    "noise_level":["1th","2th","4th"]
}
current_attributes= [zerosum_attributes,muffliato_attributes]


display_attributes = {
    "hue":"noise_level",
    "style":"variant",
    "size":"avgsteps", # For some reason having both style and size breaks in this case.
    "col":"topology_type",
}

zerosum_experiments = plot_loaders.filter_attribute_list(experiments_attributes,current_attributes)
for experiment in sorted(zerosum_experiments):
    print(experiment)



plot_utils.plot_all_experiments(
    formatted_data,
    zerosum_experiments,
    display_attributes,
    "High noise:Accuracy of Zerosum and Muffliato with and without self noise-dynamic topology",
    "test_acc mean",
    save_directory=save_directory,
    orderings=ORDERINGS,
)

plt.figure()

plot_utils.plot_all_experiments(
    formatted_data,
    zerosum_experiments,
    display_attributes,
    "High noise:Linkability attack accuracy of Zerosum and Muffliato with and without self noise-dynamic topology",
    "linkability_top1 count_percentage_success",
    save_directory=save_directory,
    orderings=ORDERINGS,
)

# Plot the middle noises
importlib.reload(plot_utils)
zerosum_attributes = {
    "variant":["nonoise","zerosum"],
    # "topology_type":["dynamic"],
    "avgsteps": ["1avgsteps"], 
    "additional_attribute": ["nonoise","selfnoise"],
    "noise_level":["nonoise","8th","16th"]
}

muffliato_attributes = {
    "variant":["muffliato"],
    # "topology_type":["dynamic"],
    "avgsteps": ["10avgsteps"], 
    "noise_level":["8th","16th"]
}
current_attributes= [zerosum_attributes,muffliato_attributes]
zerosum_experiments = plot_loaders.filter_attribute_list(experiments_attributes,current_attributes)

plt.figure()
plot_utils.plot_all_experiments(
    formatted_data,
    zerosum_experiments,
    display_attributes,
    "Middle noise:Accuracy of Zerosum and Muffliato with and without self noise-dynamic topology",
    "test_acc mean",
    save_directory=save_directory,
    orderings=ORDERINGS,
)

plt.figure()
plot_utils.plot_all_experiments(
    formatted_data,
    zerosum_experiments,
    display_attributes,
    "Middle noise:Linkability attack accuracy of Zerosum and Muffliato with and without self noise-dynamic topology",
    "linkability_top1 count_percentage_success",
    save_directory=save_directory,
    orderings=ORDERINGS,
)


# Plot the low noises
importlib.reload(plot_utils)
zerosum_attributes = {
    "variant":["nonoise","zerosum"],
    # "topology_type":["dynamic"],
    "avgsteps": ["1avgsteps"], 
    "additional_attribute": ["nonoise","selfnoise"],
    "noise_level":["nonoise","32th","64th","128th"]
}

muffliato_attributes = {
    "variant":["muffliato"],
    # "topology_type":["dynamic"],
    "avgsteps": ["10avgsteps"], 
    "noise_level":["32th","64th","128th"]
}
current_attributes= [zerosum_attributes,muffliato_attributes]
zerosum_experiments = plot_loaders.filter_attribute_list(experiments_attributes,current_attributes)

plt.figure()
plot_utils.plot_all_experiments(
    formatted_data,
    zerosum_experiments,
    display_attributes,
    "Low noise:Accuracy of Zerosum and Muffliato with and without self noise-dynamic topology",
    "test_acc mean",
    save_directory=save_directory,
    orderings=ORDERINGS,
)

plt.figure()
plot_utils.plot_all_experiments(
    formatted_data,
    zerosum_experiments,
    display_attributes,
    "Low noise:Linkability attack accuracy of Zerosum and Muffliato with and without self noise-dynamic topology",
    "linkability_top1 count_percentage_success",
    save_directory=save_directory,
    orderings=ORDERINGS,
)
plt.ylim(top=1)



# Start average trial displays

In [ ]:
importlib.reload(plot_utils)
zerosum_attributes = {
    "variant":["nonoise","zerosum"],
    # "topology_type":["dynamic"],
    "avgsteps": ["1avgsteps"], 
    "additional_attribute": ["nonoise","selfnoise"],
    "noise_level":["nonoise","4th"]
}

muffliato_attributes = {
    "variant":["muffliato"],
    # "topology_type":["dynamic"],
    "avgsteps": ["10avgsteps"], 
    "noise_level":["64th"]
}
current_attributes= [zerosum_attributes,muffliato_attributes]

display_attributes = {
    "hue":["variant","noise_level","avgsteps"],
    "style":["variant","noise_level","avgsteps"],
    "size":["variant","noise_level","avgsteps"], # For some reason having both style and size breaks in this case.
    "col":"topology_type",
}

order_mapper = {"muffliato":2,"nonoise":0,"zerosum":1}
zerosum_experiments = plot_loaders.filter_attribute_list(experiments_attributes,current_attributes)
zerosum_experiments = sorted(zerosum_experiments,key = lambda x : order_mapper[x.split("_")[1]])
for experiment in sorted(zerosum_experiments):
    print(experiment)



plot_utils.plot_all_experiments(
    formatted_data,
    zerosum_experiments,
    display_attributes,
    "Selected noise: Accuracy of Zerosum and Muffliato",
    "test_acc mean",
    save_directory=save_directory,
    orderings = ORDERINGS,
)


plt.figure()

plot_utils.plot_all_experiments(
    formatted_data,
    zerosum_experiments,
    display_attributes,
    "Selected noise: Threshold AUC rolling average of Zerosum and Muffliato",
    "roc_auc mean_start_avg",
    save_directory=save_directory,
    orderings = ORDERINGS,
)

plt.figure()

plot_utils.plot_all_experiments(
    formatted_data,
    zerosum_experiments,
    display_attributes,
    "Selected noise: Linkability attack rolling average ac of Zerosum and Muffliato",
    "linkability_top1 count_percentage_success_start_avg",
    save_directory=save_directory,
    orderings = ORDERINGS,
)

plt.figure()

#Needs to only have hue since we can't have other attributes in jointplots
display_attributes = {
    "hue":["variant","noise_level","avgsteps"],
}

zerosum_attributes = {
    "variant":["nonoise","zerosum"],
    "topology_type":["static"],
    "avgsteps": ["1avgsteps"], 
    "additional_attribute": ["nonoise","selfnoise"],
    "noise_level":["nonoise","4th"]
}

muffliato_attributes = {
    "variant":["muffliato"],
    "topology_type":["static"],
    "avgsteps": ["10avgsteps"], 
    "noise_level":["64th"]
}
current_attributes= [zerosum_attributes,muffliato_attributes]

zerosum_experiments = plot_loaders.filter_attribute_list(experiments_attributes,current_attributes)
for experiment in sorted(zerosum_experiments):
    print(experiment)
print("-"*40)


plot_utils.scatter_all_experiments(
    formatted_data,
    zerosum_experiments,
    display_attributes,
    "Selected noise: Accuracy vs Threshold AUC rolling average-static",
    "roc_auc mean_start_avg",
    "test_acc mean",
    save_directory=save_directory,
    orderings=ORDERINGS,
)
plt.figure()
plot_utils.scatter_all_experiments(
    formatted_data,
    zerosum_experiments,
    display_attributes,
    "Selected noise: Accuracy vs Linkability sucess rate rolling average-static",
    "linkability_top1 count_percentage_success_start_avg",
    "test_acc mean",
    save_directory=save_directory,
    orderings=ORDERINGS,
)

In [ ]:
last_expe = list(formatted_data.keys())[-1]
print(formatted_data[last_expe]["noise_level"])

# Display the test loss and test accuracy of inflexion point.

In [ ]:
importlib.reload(plot_utils)

muffliato_attributes = {
    "variant":["zerosum"],
    "additional_attribute":["selfnoise"],
    # "topology_type":["dynamic"],
    "avgsteps": ["1avgsteps"], 
    "noise_level":["6th",
        "7th",
        "8th",
        "16th",
        "32th","64th","128th"]
}
current_attributes= [muffliato_attributes]

display_attributes = {
    "hue":["variant","noise_level_value"],
    # "style":["variant","noise_level","avgsteps"],
    # "size":["variant","noise_level","avgsteps"], # For some reason having both style and size breaks in this case.
    "col":"topology_type",
}

muffliato_experiments = plot_loaders.filter_attribute_list(experiments_attributes,current_attributes)
for experiment in muffliato_experiments:
    print(experiment)

plot_utils.plot_all_experiments(
    formatted_data,
    muffliato_experiments,
    display_attributes,
    "Inflexion point: test_acc of Muffliato",
    "test_acc mean",
    save_directory=save_directory,
    # orderings = ORDERINGS,
)
plt.figure()
plot_utils.plot_all_experiments(
    formatted_data,
    muffliato_experiments,
    display_attributes,
    "Inflexion point: test_loss of Muffliato",
    "test_loss mean",
    save_directory=save_directory,
    # orderings = ORDERINGS,
)

# Data displays

In [ ]:
importlib.reload(plot_utils)

target_accuracy = 50


zerosum_attributes = {
    "variant":["nonoise","zerosum"],
    "topology_type":["static"],
    "avgsteps": ["1avgsteps"], 
    "additional_attribute": ["nonoise","selfnoise"],
    # "noise_level":["nonoise","4th"]
}

muffliato_attributes = {
    "variant":["muffliato"],
    "topology_type":["static"],
    "avgsteps": ["10avgsteps"], 
    # "noise_level":["64th"]
}
current_attributes= [zerosum_attributes,muffliato_attributes]

zerosum_experiments = plot_loaders.filter_attribute_list(experiments_attributes,current_attributes)
for experiment in sorted(zerosum_experiments):
    print(experiment)
order = ORDERINGS["noise_level"]
order_mapper = {"muffliato":2,"nonoise":1,"zerosum":0}
zerosum_experiments = sorted(zerosum_experiments, key = lambda x : order_mapper[x.split("_")[1]])

plot_utils.plot_communication(formatted_data,zerosum_experiments,target_accuracy,f"Communications to reach {target_accuracy}% accuracy-static",save_directory=save_directory,order=order)

zerosum_attributes = {
    "variant":["nonoise","zerosum"],
    "topology_type":["dynamic"],
    "avgsteps": ["1avgsteps"], 
    "additional_attribute": ["nonoise","selfnoise"],
    # "noise_level":["nonoise","4th"]
}

muffliato_attributes = {
    "variant":["muffliato"],
    "topology_type":["dynamic"],
    "avgsteps": ["10avgsteps"], 
    # "noise_level":["64th"]
}

current_attributes= [zerosum_attributes,muffliato_attributes]
zerosum_experiments = plot_loaders.filter_attribute_list(experiments_attributes,current_attributes)

order = ORDERINGS["noise_level"]
order_mapper = {"muffliato":2,"nonoise":0,"zerosum":1}
zerosum_experiments = sorted(zerosum_experiments, key = lambda x : order_mapper[x.split("_")[1]])

plt.figure()
plot_utils.plot_communication(formatted_data,zerosum_experiments,target_accuracy,f"Communications to reach {target_accuracy}% accuracy-dynamic",save_directory=save_directory,order=order)
